In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import unicodedata

In [2]:
url = 'https://www.cdc.gov/coronavirus/2019-ncov/cases-updates/cases-in-us.html'

### Extract date

In [3]:
content = requests.get(url).content
soup =BeautifulSoup(content, 'lxml')

In [4]:
def _removeNonAscii(s): return "".join(i for i in s if ord(i)<128)

data = soup.findAll(text=re.compile(".*pdated .*202"))
data


['Last updated on April 29, 2020']

In [5]:
if len(data)>0:
    m = re.search(".*pdated .* (\w+ \d+, 202\d)", _removeNonAscii(data[0]))
    data_date_str = m.group(1)
    data_date = datetime.strptime(data_date_str, '%B %d, %Y').strftime('%Y-%m-%d')
else:
    data_date = datetime.utcnow().strftime('%Y-%m-%d')
data_date

'2020-04-29'

### Extract data and add into Dataframe

In [6]:
dfs = pd.read_html(url)

In [7]:
if len(dfs) == 0 :
    raise Exception('There is no dataframes detected')

In [8]:
dfs

[                           Unnamed: 0_level_0  \
                             Age group (years)   
 0                                      Totals   
 1                    Race missing/unspecified   
 2                              Race specified   
 3             Among those with race specified   
 4            American Indian or Alaska Native   
 5                                       Asian   
 6                   Black or African American   
 7   Native Hawaiian or other Pacific Islander   
 8                                       White   
 9                              Multiple/other   
 10                                        NaN   
 11              Ethnicity missing/unspecified   
 12                        Ethnicity specified   
 13       Among those with ethnicity specified   
 14                            Hispanic/Latino   
 15                        Non-Hispanic/Latino   
 
                No. of cases (% of total)  \
                                     < 18   
 0      

In [9]:
df = dfs[0]
df

Unnamed: 0_level_0  \
                            Age group (years)   
0                                      Totals   
1                    Race missing/unspecified   
2                              Race specified   
3             Among those with race specified   
4            American Indian or Alaska Native   
5                                       Asian   
6                   Black or African American   
7   Native Hawaiian or other Pacific Islander   
8                                       White   
9                              Multiple/other   
10                                        NaN   
11              Ethnicity missing/unspecified   
12                        Ethnicity specified   
13       Among those with ethnicity specified   
14                            Hispanic/Latino   
15                        Non-Hispanic/Latino   

               No. of cases (% of total)  \
                                    < 18   
0                                  16275   
1                          10,902(67.0%)   
2                           5,373(33.0%)   
3        Among those with race specified   
4                               92(1.7%)   
5                              251(4.7%)   
6                           1,296(24.1%)   
7                               36(0.7%)   
8                           2,702(50.3%)   
9                             996(18.5%)   
10                                   NaN   
11                         10,828(66.5%)   
12                          5,447(33.5%)   
13  Among those with ethnicity specified   
14                          2,150(39.5%)   
15                          3,297(60.5%)   

                                          \
                                   18-44   
0                                 276707   
1                         172,508(62.3%)   
2                         104,199(37.7%)   
3        Among those with race specified   
4                              955(0.9%)   
5                            5,601(5.4%)   
6                          29,071(27.9%)   
7                              420(0.4%)   
8                          51,922(49.8%)   
9                          16,230(15.6%)   
10                                   NaN   
11                        180,344(65.2%)   
12                         96,363(34.8%)   
13  Among those with ethnicity specified   
14                         29,868(31.0%)   
15                         66,495(69.0%)   

                                          \
                                   45-64   
0                                 274124   
1                         159,238(58.1%)   
2                         114,886(41.9%)   
3        Among those with race specified   
4                              827(0.7%)   
5                            6,098(5.3%)   
6                          36,321(31.6%)   
7                              327(0.3%)   
8                          56,344(49.0%)   
9                          14,969(13.0%)   
10                                   NaN   
11                        171,125(62.4%)   
12                        102,999(37.6%)   
13  Among those with ethnicity specified   
14                         25,632(24.9%)   
15                         77,367(75.1%)   

                                          \
                                   65-74   
0                                  85248   
1                          40,955(48.0%)   
2                          44,293(52.0%)   
3        Among those with race specified   
4                              238(0.5%)   
5                            2,048(4.6%)   
6                          14,272(32.2%)   
7                              102(0.2%)   
8                          23,356(52.7%)   
9                            4,277(9.7%)   
10                                   NaN   
11                         47,354(55.5%)   
12                         37,894(44.5%)   
13  Among those with ethnicity specified   
14                          6,653(17.6%)   
15               

In [10]:
csv_file='../data/cdc-us-cases-by-races-and-age-group.csv'
df.to_csv(csv_file, index=False)

In [11]:
# col0 = df.columns[0]
# df = df.set_index(col0).T
# df



In [12]:
# # remove non alphanumeric characters from the columns
# # also lowercase them
# df.columns = df.columns.map(lambda x: re.sub('\W+', '', str.lower(x)))

# # add column date to the front
# df.insert(0, 'date', data_date)


### Save to file (old data)


In [13]:
# from pathlib import Path
# csv_file = '../data/cdc-us-cases.csv'

# if Path(csv_file).exists():
#     # read out the old dat
#     old_df = pd.read_csv(csv_file)
#     df = df.append(old_df)
# df.drop_duplicates(subset=['date'], inplace=True)

In [14]:
# df.sort_values(by=['date'], inplace=True)
# df
# df.to_csv(csv_file, index=False)